In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
from IPython.display import clear_output
from simple_order_book import SimpleOrderBook
from datetime import datetime, timedelta
import json
from bintrees import FastAVLTree
import time
import sys

DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S.%f'

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def clear_output_and_print(string_to_print):
    clear_output(wait=True)
    print(string_to_print)
        
def remove_level_if_exists(tree,level):
    if level in tree:
        del tree[level]

def update_book(book,orderbook_update, DATETIME_FORMAT):
    """takes a orderbook_update model and applies it to the book"""
    bids_overrides = json.loads(orderbook_update[8])
    asks_overrides = json.loads(orderbook_update[9])
    for bid in bids_overrides:
        price = float(bid[0])
        size = float(bid[1])
        remove_level_if_exists(book._bids,price)
        if size != 0:
            book.submit_order(order_type='lmt', side='bid', size=size, price=price, participant_id=0)
    for ask in asks_overrides:
        price = float(ask[0])
        size = float(ask[1])
        remove_level_if_exists(book._asks,price)
        if size!=0:
            book.submit_order(order_type='lmt', side='ask', size=size, price=price, participant_id=0)
    book.external_time = datetime.strptime(orderbook_update[4],DATETIME_FORMAT)
    
def get_vwap(book,vwap_depth):
    vwap_lb = book[book[:,1].cumsum() < vwap_depth]
    vwap_ub = book[book[:,1].cumsum() >= vwap_depth]
    if vwap_ub.size ==0: # vwap could not be resolved due to insufficient depth
        return np.nan
    vwap_lb_val = np.dot(vwap_lb[:,1],vwap_lb[:,0])
    vwap_ub_val = (vwap_depth - vwap_lb[:,1].sum()) * vwap_ub[0,0]
    vwap = (vwap_lb_val + vwap_ub_val)/vwap_depth
    return vwap

def generate_vwaps_list(order_book, vwap_depths_usd, DEPTH_LEVELS_TO_GET):
    both_sides = order_book.get_mkt_depth(DEPTH_LEVELS_TO_GET)
    bids_np = np.array(both_sides[1])
    asks_np = np.array(both_sides[0])
    bids_size = bids_np.sum(axis=0)[1]
    asks_size = asks_np.sum(axis=0)[1]
    mid_rate = (order_book.bid + order_book.ask)/2
    current_vwap_snaps = []
    for vwap_depth_usd in vwap_depths_usd:
        vwap_depth_ccy = vwap_depth_usd/mid_rate
        bid_vwap = get_vwap(bids_np,vwap_depth_ccy)
        ask_vwap = get_vwap(asks_np,vwap_depth_ccy)
        current_vwap_snaps.append({
            'orderbook_snapshot_id':order_book.orderbook_snapshot_id,
            'external_time':order_book.external_time,
            'vwap_depth_usd':vwap_depth_usd,
            'bid_vwap':bid_vwap,
            'ask_vwap':ask_vwap,
            'top_bid':order_book.bid,
            'top_ask':order_book.ask,
            'total_ask_size':order_book._total_ask_size,
            'total_bid_size':order_book._total_bid_size,
            'bids_size':bids_size,
            'asks_size':asks_size,
            'levels_extracted':DEPTH_LEVELS_TO_GET,
                        })
    return current_vwap_snaps

def load_book_from_snapshot(orderbook_snapshot):
    provider = orderbook_snapshot[4]
    internal_pair = f"{orderbook_snapshot[2]}-{orderbook_snapshot[3]}" # columns selected external_time,base,counter,provider,bids,asks
    order_book = SimpleOrderBook()
    for bid in json.loads(orderbook_snapshot[5]):
        order_book.apply_update( side='bid', size=float(bid[1]), price=float(bid[0]))
    for ask in json.loads(orderbook_snapshot[6]):
        order_book.apply_update( side='ask', size=float(ask[1]), price=float(ask[0]))
        
    book_time = datetime.strptime(orderbook_snapshot[1],DATETIME_FORMAT)
    order_book.external_time = book_time
    order_book.orderbook_snapshot_id = orderbook_snapshot[0]
    order_book.base = orderbook_snapshot[2]
    order_book.counter = orderbook_snapshot[3]
    order_book.provider = provider
    return order_book

### Get book snapshots

In [2]:
conn = create_connection('market_data.sqlite')
cur = conn.cursor()
# columns selected orderbook_snapshot_id, external_time,base,counter,provider,bids,asks
cur.execute(f"""
with all_books as (
SELECT 
                count(*) over (partition by base,counter,provider order by external_time desc) as rnum,
                s.*
                FROM orderbook_snapshots s
                where true
                )
                select 
                id as orderbook_snapshot_id, external_time,base,counter,provider,bids,asks, rnum, base ||'_' || counter || '_'|| provider as book_key
                from all_books
                where rnum = 1 -- get latest book before time we are interested in
order by external_time
                
                
                
                """)
orderbook_snapshots_list = cur.fetchall()
orderbook_snapshots_names = list(map(lambda x: x[0], cur.description))
orderbook_snapshots_df = pd.DataFrame(orderbook_snapshots_list,columns=orderbook_snapshots_names)


### Load book snapshots

In [3]:
# %%timeit
bookid_to_book_map = {}
bookid_last_vwap_time_map = {}
for counter, orderbook_snapshot in enumerate(orderbook_snapshots_list):
    order_book = load_book_from_snapshot(orderbook_snapshot)
    bookid_last_vwap_time_map[orderbook_snapshot[0]] = datetime(2000,1,1) # init to old value
    bookid_to_book_map[orderbook_snapshot[0]] = order_book    
#     clear_output_and_print(f"done {counter+1}/{len(orderbook_snapshots_list)}. {orderbook_snapshot[2]}-{orderbook_snapshot[3]} {orderbook_snapshot[4]}. {orderbook_snapshot[1]}")

In [4]:

DEPTH_LEVELS_TO_GET = 200
VWAP_STORE_EVERY_X_SECONDS = 1
VWAP_DEPTHS_USD = [50,100,200,300,400,500,750,1_000,5_000,10_000,20_000,30_000,40_000,50_000,75_000,100_000,200_000,300_000,400_000,500_000]

In [5]:
mid_rates = {"BTC-USD":30000}

In [6]:

def compute_vwaps_list(order_book,VWAP_DEPTHS_USD):
    vwaps_computed_list = []
    
    # asks
    vwaps_computed_asks = {i:0 for i in VWAP_DEPTHS_USD}
    vwap_left = {i:i for i in VWAP_DEPTHS_USD}
    mid_rate_book = mid_rates.get(f"{order_book.base}-{order_book.counter}")
    for price, qty in order_book._asks.iter_items():
        qty_usd = qty *  mid_rate_book
        for depth_usd, depth_still_to_fill in vwap_left.copy().items():
            if depth_usd ==0:
                vwap_left.pop(depth_usd)
                vwaps_computed_asks[depth_usd] = price
            else:
                prev_qty = vwap_left[depth_usd]
                vwap_left[depth_usd] -= qty_usd
                if vwap_left[depth_usd] <= 0:
                    vwap_left.pop(depth_usd)
                    vwaps_computed_asks[depth_usd] += prev_qty*price
                    vwaps_computed_asks[depth_usd] = vwaps_computed_asks[depth_usd]/depth_usd
                else:
                    vwaps_computed_asks[depth_usd] += qty_usd*price
        if not vwap_left:
            break
            
    # bids
    vwaps_computed_bids = {i:0 for i in VWAP_DEPTHS_USD}
    vwap_left = {i:i for i in VWAP_DEPTHS_USD}
    for price, qty in order_book._bids.iter_items(reverse=True):
        qty_usd = qty *  mid_rate_book
        for depth_usd, depth_still_to_fill in vwap_left.copy().items():
            if depth_usd ==0:
                vwap_left.pop(depth_usd)
                vwaps_computed_bids[depth_usd] = price
            else:
                prev_qty = vwap_left[depth_usd]
                vwap_left[depth_usd] -= qty_usd
                if vwap_left[depth_usd] <= 0:
                    vwap_left.pop(depth_usd)
                    vwaps_computed_bids[depth_usd] += prev_qty*price
                    vwaps_computed_bids[depth_usd] = vwaps_computed_bids[depth_usd]/depth_usd
                else:
                    vwaps_computed_bids[depth_usd] += qty_usd*price
        if not vwap_left:
            break
    for depth_usd in VWAP_DEPTHS_USD:
        vwap_ask = vwaps_computed_asks.get(depth_usd)
        vwap_bid = vwaps_computed_bids.get(depth_usd)
        if (vwap_ask or vwap_bid):
            vwaps_computed_list.append({'bid':vwap_bid,'ask':vwap_ask,'depth_usd':depth_usd})
            
    return vwaps_computed_list

In [7]:
%%timeit
compute_vwaps_list(order_book,VWAP_DEPTHS_USD)

242 µs ± 36.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
order_book

In [12]:
# compute_vwaps_list(order_book,VWAP_DEPTHS_USD)